# 분류 결과

### [경제,기업] 기사를 제일 못맞춘다. 정확도가 50%수준..

### 문화 기사와 부동산, 증권 기사는 잘 맞춘다. 

In [19]:
result_df = pd.DataFrame(confusion_matrix(y_test, y_pred) ,
                         index = sorted(train_df['Section'].unique()),
                         columns = sorted(train_df['Section'].unique()))

result_df

,IT과학,경제,국제,기업,문화,부동산,사회,정치,증권
IT과학,581,38,28,210,29,9,48,7,50
경제,59,493,50,114,10,35,47,26,166
국제,27,64,759,28,25,6,24,40,27
기업,191,80,21,497,30,15,62,19,85
문화,23,11,15,13,818,14,73,28,5
부동산,15,38,2,16,11,844,46,19,9
사회,44,35,18,46,36,45,715,50,11
정치,5,47,55,14,18,8,87,760,6
증권,50,50,4,59,3,10,3,0,821


#### 특징 1. IT과학 기사는 기업 기사와 가장 많이 혼동한다.
 - 예상 원인: 기업과 IT과학 기사에 기업의 이름이 너무 빈번하게 등장하기 때문..?



#### 특징 2. 경제 기사는 [증권, 기업] 기사와 가장 많이 혼동한다.
 - 예상 원인:  비슷한 카테고리, 하지만 대부분 다 증권기사라고 분류해버림.. Doc2Vec에서 vector space에 잘 span하지 못했을 것?

#### 특징 4. [증권 기업 경제]와 달리 [정치 사회] 기사는 유사한 카테고리임에도 정확도가 우수한 편에 속한다. 
 - 예상 원인: ???

## 너무 많은 수의 기사를 [사회, 증권] 기사라고 분류한다.

## 반대로　경제　기사는　오히려　다른　것으로　자꾸　분류해버림

In [20]:
result_df.sum(axis = 0)

IT과학     995
경제       856
국제       952
기업       997
문화       980
부동산      986
사회      1105
정치       949
증권      1180
dtype: int64

<br> 


## 각 기사의 토큰 수에 따른 분류 정확도

In [114]:
pd.DataFrame(store_dict)

,Token의 수 n개 이상,Token의 수 n개 이하,경제,기업,사회,국제,부동산,증권,정치,IT과학,문화
0,50,100,54.0 %,40.0 %,81.0 %,62.0 %,80.0 %,93.0 %,64.0 %,36.0 %,74.0 %
1,100,300,46.0 %,54.0 %,73.0 %,76.0 %,86.0 %,82.0 %,74.0 %,60.0 %,79.0 %
2,300,500,47.0 %,59.0 %,76.0 %,76.0 %,87.0 %,82.0 %,69.0 %,64.0 %,80.0 %
3,500,10000,49.0 %,59.0 %,73.0 %,74.0 %,88.0 %,82.0 %,71.0 %,63.0 %,81.0 %


#### 길이가 짧으면 **[증권, 사회]** 기사는 매우 잘 분류하는 반면, **[정치, IT과학, 국제]** 기사는 잘 분류하지 못한다.

#### 길이가 길어지면 반대가 됨!

### **그 이유는, 문서 길이에 따라 문서의 카테고리를 다르게 추정해버리기 때문이라고 생각됨..!**

#### 길이가 짧은 문서들은 전부 [증권, 사회] 기사로 분류해버리고, 길이가 긴 문서들은 [정치, 부동산, 문화] 기사로 분류하기 때문?

# 왜 그럴까?

# 각 카테고리 별, 토큰 길이의 평균은?

In [27]:
train_df.groupby('Section')['Num of Tokens'].describe()

,count,mean,std,min,25%,50%,75%,max
Section,,,,,,,,
IT과학,9836.0,228.367222,150.858785,51.0,133.0,191.0,279.0,2606.0
경제,8050.0,215.267702,144.405833,51.0,114.0,174.0,279.0,2129.0
국제,9677.0,244.187765,131.619560,51.0,154.0,219.0,300.0,1342.0
기업,9604.0,204.217618,137.728449,51.0,119.0,171.0,250.0,2416.0
문화,9313.0,248.453452,258.583833,51.0,112.0,176.0,314.0,4327.0
부동산,9714.0,224.478691,136.277920,51.0,130.0,188.0,277.0,1302.0
사회,9325.0,187.522681,121.342611,51.0,107.0,156.0,232.0,2020.0
정치,8875.0,240.445746,147.590659,51.0,134.0,203.0,313.0,1776.0
증권,8569.0,188.156261,103.081897,51.0,109.0,178.0,251.0,1379.0


#### 평균은 비슷한 것 같은데.. [경제 기업 사회 증권] 기사는 대체적으로 좀 짧고 (평균이 220 정도에, 1Q가 1２0 이하) 

#### 나머지 기사들은 조금 긴편에 속하는 것 같다. (평균은 250정도에 1Q가 140 이상) 

# 추측 1. 

# [경제 사회 증권] 기사는 대체적으로 짧기 때문에, 짧은 기사들은 [경제 사회 증권]이라고 분류한다.

### 다시 살펴보면, [경제 사회 증권] 기사는 토큰 수가 많아질수록 분류 성능이 떨어짐

### 기업은 예외적으로 성능이 향상

In [26]:
pd.DataFrame(store_dict)

,Token의 수 n개 이상,Token의 수 n개 이하,경제,기업,사회,국제,부동산,증권,정치,IT과학,문화
0,50,100,46.0 %,36.0 %,75.0 %,50.0 %,75.0 %,92.0 %,60.0 %,31.0 %,71.0 %
1,100,300,43.0 %,53.0 %,76.0 %,80.0 %,84.0 %,80.0 %,70.0 %,63.0 %,81.0 %
2,300,500,42.0 %,57.0 %,73.0 %,77.0 %,86.0 %,82.0 %,70.0 %,65.0 %,83.0 %
3,500,10000,44.0 %,54.0 %,75.0 %,70.0 %,87.0 %,81.0 %,68.0 %,62.0 %,82.0 %
